In [57]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from patchify import patchify

In [58]:
# Функция для удаления черных краев
def full_remove_black_borders(image):
    """
    Удаляет черные края вокруг изображения, оставляя только полезную область.
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return image  # Если контуры не найдены, возвращаем исходное изображение
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    cropped_image = image[y:y + h, x:x + w]
    return cropped_image

In [59]:
# 2. Создание структуры директорий
base_dir = "patched_dataset"
subdirs = ["train_images/train", "train_masks/train", "val_images/val", "val_masks/val"]

for subdir in subdirs:
    os.makedirs(os.path.join(base_dir, subdir), exist_ok=True)

In [60]:
# Функция для обработки и сохранения патчей
def process_and_save_patches(image_dir, mask_dir, output_image_dir, output_mask_dir, patch_size):
    """
    Обрезает черные края только у изображений, нарезает патчи и сохраняет их.
    """
    image_paths = [os.path.join(image_dir, fname) for fname in os.listdir(image_dir)]
    mask_paths = [os.path.join(mask_dir, fname) for fname in os.listdir(mask_dir)]

    for img_path, mask_path in zip(image_paths, mask_paths):
        image = cv2.imread(img_path)
        mask = cv2.imread(mask_path, 0)  # Маска в градациях серого

        # Убедимся, что размеры изображения и маски совпадают
        if image.shape[:2] != mask.shape:
            print(f"Размеры изображения и маски не совпадают: {img_path}, {mask_path}")
            continue

        # Удаление черных краев только у изображения
        try:
            image = full_remove_black_borders(image)
        except ValueError as e:
            print(f"Ошибка при обработке черных краев: {img_path} — {str(e)}")
            continue

        # Добавляем отступы, чтобы размеры делились на patch_size
        pad_h = (patch_size - image.shape[0] % patch_size) % patch_size
        pad_w = (patch_size - image.shape[1] % patch_size) % patch_size

        image = cv2.copyMakeBorder(image, 0, pad_h, 0, pad_w, cv2.BORDER_CONSTANT, value=[0, 0, 0])
        mask = cv2.copyMakeBorder(mask, 0, pad_h, 0, pad_w, cv2.BORDER_CONSTANT, value=0)

        # Нарезаем патчи
        image_patches = patchify(image, (patch_size, patch_size, 3), step=patch_size)
        mask_patches = patchify(mask, (patch_size, patch_size), step=patch_size)

        # Сохраняем патчи
        for i in range(image_patches.shape[0]):
            for j in range(image_patches.shape[1]):
                img_patch = image_patches[i, j, 0]
                mask_patch = mask_patches[i, j]

                img_patch_name = f"{os.path.basename(img_path)[:-4]}_patch_{i}_{j}.png"
                mask_patch_name = f"{os.path.basename(mask_path)[:-4]}_patch_{i}_{j}.png"

                cv2.imwrite(os.path.join(output_image_dir, img_patch_name), img_patch)
                cv2.imwrite(os.path.join(output_mask_dir, mask_patch_name), mask_patch)

# Пути к данным
train_image_dir = "dataset/train_images/train"
train_mask_dir = "dataset/train_masks/train"
val_image_dir = "dataset/val_images/val"
val_mask_dir = "dataset/val_masks/val"

# Параметры
base_dir = "patched_dataset"
patch_size = 128

os.makedirs(os.path.join(base_dir, "train_images/train"), exist_ok=True)
os.makedirs(os.path.join(base_dir, "train_masks/train"), exist_ok=True)
os.makedirs(os.path.join(base_dir, "val_images/val"), exist_ok=True)
os.makedirs(os.path.join(base_dir, "val_masks/val"), exist_ok=True)

# Обрабатываем тренировочные и валидационные наборы
process_and_save_patches(train_image_dir, train_mask_dir, os.path.join(base_dir, "train_images/train"), os.path.join(base_dir, "train_masks/train"), patch_size)
process_and_save_patches(val_image_dir, val_mask_dir, os.path.join(base_dir, "val_images/val"), os.path.join(base_dir, "val_masks/val"), patch_size)


In [61]:
# Функция для визуализации случайного патча
def visualize_random_patch(image_dir, mask_dir):
    """
    Визуализирует случайный патч изображения и его маску.
    """
    print(f"Чтение из папок:\n- Изображения: {image_dir}\n- Маски: {mask_dir}")
    
    image_files = os.listdir(image_dir)
    if not image_files:
        raise ValueError(f"Папка {image_dir} пуста.")
    
    random_file = np.random.choice(image_files)
    print(f"Выбран файл: {random_file}")
    
    image_path = os.path.join(image_dir, random_file)
    mask_path = os.path.join(mask_dir, random_file)
    
    if not os.path.exists(mask_path):
        raise ValueError(f"Маска {random_file} отсутствует в {mask_dir}.")

    # Чтение изображения и маски
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, 0)  # Маска читается в градациях серого

    if image is None:
        raise ValueError(f"Изображение {random_file} не может быть прочитано.")
    if mask is None:
        raise ValueError(f"Маска {random_file} не может быть прочитана.")

    print(f"Размеры изображения: {image.shape}, маски: {mask.shape}")

    # Отображение
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    ax[0].set_title("Original Image Patch")
    ax[1].imshow(mask, cmap="gray")
    ax[1].set_title("Mask Patch")
    overlay = image.copy()
    overlay[mask > 0] = [255, 0, 0]
    ax[2].imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
    ax[2].set_title("Overlay")
    plt.show()

In [62]:
# Проверка на тренировочном наборе
visualize_random_patch(os.path.join(base_dir, "train_images/train"), os.path.join(base_dir, "train_masks/train"))

Чтение из папок:
- Изображения: patched_dataset\train_images/train
- Маски: patched_dataset\train_masks/train
Выбран файл: 030_43-19-ROOT1-2023-08-08_pvdCherry_OD001_Col0_04-Fish Eye Corrected_patch_17_7.png


ValueError: Маска 030_43-19-ROOT1-2023-08-08_pvdCherry_OD001_Col0_04-Fish Eye Corrected_patch_17_7.png отсутствует в patched_dataset\train_masks/train.